# Remove not solidity file

In [21]:
import pandas as pd

In [24]:
train_file = pd.read_parquet("./data/solfile/train_file.parquet", engine="fastparquet")
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167758 entries, 0 to 167757
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   file_name         167758 non-null  object
 1   file_address      167758 non-null  object
 2   source_code       167758 non-null  object
 3   abi               167758 non-null  object
 4   compiler_version  167758 non-null  object
 5   library           167758 non-null  object
 6   language          167758 non-null  object
dtypes: object(7)
memory usage: 9.0+ MB


In [25]:
test_file = pd.read_parquet("./data/solfile/test_file.parquet", engine="fastparquet")
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18639 entries, 0 to 18638
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   file_name         18639 non-null  object
 1   file_address      18639 non-null  object
 2   source_code       18639 non-null  object
 3   abi               18639 non-null  object
 4   compiler_version  18639 non-null  object
 5   library           18639 non-null  object
 6   language          18639 non-null  object
dtypes: object(7)
memory usage: 1019.4+ KB


In [26]:
train_file = train_file[train_file["language"] == "Solidity"]
test_file = test_file[test_file["language"] == "Solidity"]
train_file.reset_index(drop=True, inplace=True)
test_file.reset_index(drop=True, inplace=True)
train_file = train_file.drop(columns=["language"])
test_file = test_file.drop(columns=["language"])
train_file.to_parquet("./data/solfile/train_file.parquet", engine="fastparquet")
test_file.to_parquet("./data/solfile/test_file.parquet", engine="fastparquet")

# Check result

In [4]:
train_file = pd.read_parquet("./data/solfile/train_file.parquet", engine="fastparquet")
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167310 entries, 0 to 167309
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   file_name         167310 non-null  object
 1   file_address      167310 non-null  object
 2   source_code       167310 non-null  object
 3   abi               167310 non-null  object
 4   compiler_version  167310 non-null  object
 5   library           167310 non-null  object
dtypes: object(6)
memory usage: 7.7+ MB


In [5]:
test_file = pd.read_parquet("./data/solfile/test_file.parquet", engine="fastparquet")
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18594 entries, 0 to 18593
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   file_name         18594 non-null  object
 1   file_address      18594 non-null  object
 2   source_code       18594 non-null  object
 3   abi               18594 non-null  object
 4   compiler_version  18594 non-null  object
 5   library           18594 non-null  object
dtypes: object(6)
memory usage: 871.7+ KB


# Experiment

In [6]:
versions = list(set(train_file["compiler_version"].tolist() + test_file["compiler_version"].tolist()))
versions = list(set([version.split('+')[0][1:].split('-')[0] for version in versions]))
string = ""
for version in versions:
    string = string + '{' + f'version: "{version}"'+ '},\n'
with open("sol_version.txt", "w") as f:
    f.write(string)

In [21]:
from subprocess import run 

for i in range(len(test_file)):
    source = test_file.loc[i, "source_code"]
    with open("./hardhat/contracts/sample.sol", "w") as f:
        f.write(source)
    cmd = """
    cd hardhat
    npx hardhat compile
    """
    data = run(cmd, capture_output=True, shell=True, text=True)
    # print(data.stdout)
    # print("-" * 400)
    # print(data.stderr)
    # print(test_file.loc[i, "file_address"])
    if 'Compiled 1 Solidity file successfully' in data.stdout:
        with open("compilable.txt", "a") as f:
            f.write(f"{i}\n---------------------------------------------------\n")
    



----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
ParserError: Expected ';' but got '}'
  --> contracts/sample.sol:11:60:
   |
11 |     modifier onlyOwner { if (msg.sender != owner) throw; _ }
   |                                                            ^


Error HH600: Compilation failed
For more info go to https://hardhat.org/HH600 or run Hardhat with --show-stack-traces

0x6abd2b75ff5f306a4d99bfab1ff84b57bb9d23e7


In [3]:
import pandas as pd
ori_train_file = pd.read_parquet("./data/solfile/ori_train_file.parquet", engine="fastparquet")
ori_train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167758 entries, 0 to 167757
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   contract_name          167758 non-null  object
 1   file_path              167758 non-null  object
 2   contract_address       167758 non-null  object
 3   language               167758 non-null  object
 4   source_code            167758 non-null  object
 5   abi                    167758 non-null  object
 6   compiler_version       167758 non-null  object
 7   optimization_used      167758 non-null  bool  
 8   runs                   167758 non-null  int64 
 9   constructor_arguments  167758 non-null  object
 10  evm_version            167758 non-null  object
 11  library                167758 non-null  object
 12  license_type           167758 non-null  object
 13  proxy                  167758 non-null  bool  
 14  implementation         167758 non-null  object
 15  

In [8]:
ori_test_file = pd.read_parquet("./data/solfile/ori_test_file.parquet", engine="fastparquet")
ori_test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18639 entries, 0 to 18638
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   contract_name          18639 non-null  object
 1   file_path              18639 non-null  object
 2   contract_address       18639 non-null  object
 3   language               18639 non-null  object
 4   source_code            18639 non-null  object
 5   abi                    18639 non-null  object
 6   compiler_version       18639 non-null  object
 7   optimization_used      18639 non-null  bool  
 8   runs                   18639 non-null  int64 
 9   constructor_arguments  18639 non-null  object
 10  evm_version            18639 non-null  object
 11  library                18639 non-null  object
 12  license_type           18639 non-null  object
 13  proxy                  18639 non-null  bool  
 14  implementation         18639 non-null  object
 15  swarm_source       

In [10]:
ori_test_file[ori_test_file["contract_address"] == "0xb374387a340e6aa7d78385c4a4aac6b425a685b0"].loc[3, "source_code"]

'contract Strategy is BaseStrategyInitializable {\n    using SafeERC20 for IERC20;\n    using Address for address;\n    using SafeMath for uint256;\n\n    IUniswapV2Router02 constant public uniswapRouter = IUniswapV2Router02(address(0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D));\n    IUniswapV2Router02 constant public sushiswapRouter = IUniswapV2Router02(address(0xd9e1cE17f2641f24aE83637ab66a2cca9C378B9F));\n    IERC20 public constant weth = IERC20(address(0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2));\n    IERC20 public constant rook = IERC20(address(0xfA5047c9c78B8877af97BDcb85Db743fD7313d4a));\n    IUniswapV2Router02 public router;\n    IDistributeV1 public distributor;\n    IKToken public kToken;\n    ILiquidityPool public pool;\n    address public treasury;\n    address[] public path;\n    address[] public wethWantPath;\n\n    // unsigned. Indicates the losses incurred from the protocol\'s deposit fees\n    uint256 public incurredLosses;\n    // amount to send to treasury. Used for 